# Résumé automatique de documents financiers

Objectif : transformer rapidement un **rapport financier** (annuel, trimestriel, comptes, bilan, annexes) en un **résumé clair et chiffré**.

# Résumé automatique de documents financiers (PDF)


### PARTIE 1 — Préparation (imports, .env)
- Charger les librairies nécessaires  
- Lire les variables depuis `.env` (clé API, chemin du PDF, modèle OpenAI)  
- Vérifier la configuration minimale  

### PARTIE 2 — Lecture du PDF (avec repères par page)
- Ouvrir le fichier PDF avec **PyMuPDF**  
- Extraire le texte page par page  
- Ajouter un séparateur clair `=== [PAGE X] ===`  
- Nettoyer le texte et le tronquer si trop long  

### PARTIE 3 — Consignes orientées FINANCE
- Définir le rôle : *analyste financier*  
- Exiger un **résumé structuré** (Société, Résumé exécutif, Chiffres clés, Analyse, Références internes)  
- Imposer la règle : **ne pas inventer de chiffres** (mettre `non précisé` si absent)  
- Demander un tableau de 6 à 12 indicateurs maximum  

### PARTIE 4 — Appel à l’API OpenAI (Responses)
- Envoyer deux messages (system + user) à l’API  
- Utiliser `client.responses.create(...)`  
- Récupérer le texte résumé depuis `response.output_text`  

### PARTIE 5 — Affichage du résumé 
- Afficher le résumé brut dans la console  
- Afficher le résumé en Markdown (plus lisible dans Jupyter)  
- Ajouter un rappel : **toujours vérifier les chiffres et pages d’origine**  


In [ ]:
#Prérequis (exécuter une seule fois dans le notebook) 
!pip install openai PyMuPDF python-dotenv

# PARTIE 1 — PRÉPARATION (imports, .env)

In [2]:
import os
import fitz # PyMuPDF
from dotenv import load_dotenv
from IPython.display import Markdown, display

In [3]:
import os, pathlib
print("Dossier courant :", pathlib.Path.cwd())


Dossier courant : /Users/natachanjongwayepnga/Documents/GENAI/02_Application_Analyseur_Financier_OpenSource_OpenRouter


In [4]:

# Chercher et charger le .env (remonte les dossiers si besoin)
from dotenv import load_dotenv, find_dotenv

env_path = find_dotenv(filename=".env", usecwd=True)
env_path

'/Users/natachanjongwayepnga/Documents/GENAI/02_Application_Analyseur_Financier_OpenSource_OpenRouter/.env'

In [5]:

load_dotenv(dotenv_path=env_path, override=True)              # Charge explicitement le fichier .env et écrase les variables existantes si nécessaire
  
import requests
from dotenv import load_dotenv
import os

# Chargement des variables d'environnement

api_key = os.getenv('OPENROUTER_API_KEY')
print(api_key[:4] + "..." + "****")  
  


sk-o...****


# Partie 2: Lecture du PDF (avec repères par page)

In [6]:

chemin_pdf = "./data/teslafinancialreport.pdf"

In [7]:

pdf = fitz.open(chemin_pdf)

texte = ""
for i, page in enumerate(pdf, start=1):
    texte_page = page.get_text()
    texte += f"\n\n=== [PAGE {i}] ===\n" + texte_page.strip()

print(texte)



=== [PAGE 1] ===
Q1 2023 Update
1

=== [PAGE 2] ===
Highlights 
03
Financial Summary
04
Operational Summary
06
Vehicle Capacity 
07
Core Technology
08
Other Highlights
09
Outlook
10
Photos & Charts 
11
Key Metrics
19
Financial Statements
22
Additional Information
28

=== [PAGE 3] ===
S U M M A R Y
H I G H L I G H T S  
(1) Excludes SBC (stock-based compensation).
(2) Free cash flow = operating cash flow less capex.
(3) Includes cash, cash equivalents and investments.
Profitability
11.4% operating margin in Q1
$2.7B GAAP operating income in Q1
$2.5B GAAP net income in Q1
$2.9B non-GAAP net income1 in Q1
In the current macroeconomic environment, we see this year as a unique 
opportunity for Tesla. As many carmakers are working through challenges with the 
unit economics of their EV programs, we aim to leverage our position as a cost 
leader. We are focused on rapidly growing production, investments in autonomy 
and vehicle software, and remaining on track with our growth investments.
O

In [8]:
# Nettoyage léger : retirer espaces inutiles
texte = "\n".join(l.strip() for l in texte.splitlines())
texte

'\n\n=== [PAGE 1] ===\nQ1 2023 Update\n1\n\n=== [PAGE 2] ===\nHighlights\n03\nFinancial Summary\n04\nOperational Summary\n06\nVehicle Capacity\n07\nCore Technology\n08\nOther Highlights\n09\nOutlook\n10\nPhotos & Charts\n11\nKey Metrics\n19\nFinancial Statements\n22\nAdditional Information\n28\n\n=== [PAGE 3] ===\nS U M M A R Y\nH I G H L I G H T S\n(1) Excludes SBC (stock-based compensation).\n(2) Free cash flow = operating cash flow less capex.\n(3) Includes cash, cash equivalents and investments.\nProfitability\n11.4% operating margin in Q1\n$2.7B GAAP operating income in Q1\n$2.5B GAAP net income in Q1\n$2.9B non-GAAP net income1 in Q1\nIn the current macroeconomic environment, we see this year as a unique\nopportunity for Tesla. As many carmakers are working through challenges with the\nunit economics of their EV programs, we aim to leverage our position as a cost\nleader. We are focused on rapidly growing production, investments in autonomy\nand vehicle software, and remaining on

In [9]:
# Option de sécurité : éviter d'envoyer un texte excessivement long
LONGUEUR_MAX = 120_000 # augmentez si nécessaire
if len(texte) > LONGUEUR_MAX:
    texte = texte[:LONGUEUR_MAX]

In [10]:
# Aperçu rapide (facultatif) :
print("Aperçu du début du texte :\n")
print(texte[:800])

Aperçu du début du texte :



=== [PAGE 1] ===
Q1 2023 Update
1

=== [PAGE 2] ===
Highlights
03
Financial Summary
04
Operational Summary
06
Vehicle Capacity
07
Core Technology
08
Other Highlights
09
Outlook
10
Photos & Charts
11
Key Metrics
19
Financial Statements
22
Additional Information
28

=== [PAGE 3] ===
S U M M A R Y
H I G H L I G H T S
(1) Excludes SBC (stock-based compensation).
(2) Free cash flow = operating cash flow less capex.
(3) Includes cash, cash equivalents and investments.
Profitability
11.4% operating margin in Q1
$2.7B GAAP operating income in Q1
$2.5B GAAP net income in Q1
$2.9B non-GAAP net income1 in Q1
In the current macroeconomic environment, we see this year as a unique
opportunity for Tesla. As many carmakers are working through challenges with the
unit economics of their EV programs, we a


# PARTIE 3 — Consignes (format de sortie)

In [11]:
consignes = (
"Tu es analyste financier. On te fournit le texte d'un document financier\n"
"(rapport annuel, trimestriel, comptes, bilan, annexes).\n\n"
"Produis une synthèse **précise et chiffrée** en Markdown selon ce cadre :\n\n"
"- **Société / Période / Devise** : (si repérable)\n"
"- **Résumé exécutif (5–8 lignes)** : activité, faits marquants, contexte\n"
"- **Chiffres clés** (tableau) :\n"
" | Indicateur | Valeur | Évolution/Contexte | Période | Page |\n"
" |---|---:|---|---|---:|\n"
" (exemples : Chiffre d'affaires, EBIT/EBITDA, Résultat net, Marge, FCF, CAPEX,\n"
" Dette nette, Trésorerie, NPL/Coût du risque pour banque, CET1, LCR/NSFR, etc.)\n"
"- **Analyse** :\n"
" - Performance (croissance, marges, cash)\n"
" - Structure financière (dette, liquidité)\n"
" - Risques & incertitudes (marché, réglementation, change)\n"
" - Outlook / Guidance (si communiqué)\n"
"- **Références internes** : pages/sections à relire\n\n"
"Exigences :\n"
"- **N'invente aucun chiffre**. Si une valeur n'apparaît pas clairement : `non précisé`.\n"
"- Cite la **Page** d'origine quand c'est possible (repère `=== [PAGE X] ===`).\n"
"- 6 à 12 **indicateurs quantitatifs** maximum (les plus utiles).\n"
"- Reste concis : 200–350 mots hors tableau."
)

PARTIE 4 — Appel API OpenAI (Responses)

In [12]:


# Configuration pour OpenRouter
headers = {
    "Authorization": f"Bearer {api_key}",
    "HTTP-Referer": "http://localhost:8888/", # Remplacer par votre URL
    "Content-Type": "application/json"
}

# URL de l'API OpenRouter
api_url = "https://openrouter.ai/api/v1/chat/completions"

# Modèle gratuit d'OpenRouter
modele = "mistralai/mistral-7b-instruct" # ou autre modèle gratuit disponible

# Préparation de la requête
payload = {
    "model": modele,
    "messages": [
        {"role": "system", "content": consignes},
        {"role": "user", "content": texte}
    ]
}

# Appel API
reponse = requests.post(api_url, json=payload, headers=headers)
reponse_json = reponse.json()

# Extraction du texte de la réponse
resume = reponse_json['choices'][0]['message']['content']




In [13]:
resume

" **Société / Période / Devise :** Tesla, Q1 2023, USD\n\n**Résumé exécutif (5-8 lignes)** : Tesla a annoncé son rapport de première trimestre de 2023 avec une baisse de ses marges même avec des prix plus faibles dans la plupart de ses modèles. L'entreprise est passée d'un taux de marge d'opération de 25% à 19,3% et a entamé la production de sa nouvelle fabrique de batteries afin de mieux répondre à la croissante demande en énergie et reprendre sa mission de créer un véhicule durable.\n\n**Chiffres clés** (tableau) :\n\n| Indicateur | Valeur | Évolution/Contexte | Période | Page |\n| --- | --- | --- | --- | --- |\n| Chiffre d'affaires | 23,3B$ | -24% YoY | Q1-2023 | 23 |\n| EBIT (Opérationnel) | 2,664M$ | -26% YoY | Q1-2023 | 23 |\n| Résultat net (GAAP) | 2,513M$ | -24% YoY | Q1-2023 | 23 |\n| Résultat net (Non-GAAP) | 2,931M$ | -22% YoY | Q1-2023 | 26 |\n| Cash & Cash équivalents | 22,402 | 24% YoY | Q1-2023 | 24 |\n| Capital Expenditure (CAPEX) | -2,072 | 17% YoY | Q1-2023 | 25 |\n| 

# Partie 5: Affichage du résumé (markdown)

In [14]:


print("\n=== RÉSUMÉ FINANCIER GÉNÉRÉ ===\n")
print(resume)


=== RÉSUMÉ FINANCIER GÉNÉRÉ ===

 **Société / Période / Devise :** Tesla, Q1 2023, USD

**Résumé exécutif (5-8 lignes)** : Tesla a annoncé son rapport de première trimestre de 2023 avec une baisse de ses marges même avec des prix plus faibles dans la plupart de ses modèles. L'entreprise est passée d'un taux de marge d'opération de 25% à 19,3% et a entamé la production de sa nouvelle fabrique de batteries afin de mieux répondre à la croissante demande en énergie et reprendre sa mission de créer un véhicule durable.

**Chiffres clés** (tableau) :

| Indicateur | Valeur | Évolution/Contexte | Période | Page |
| --- | --- | --- | --- | --- |
| Chiffre d'affaires | 23,3B$ | -24% YoY | Q1-2023 | 23 |
| EBIT (Opérationnel) | 2,664M$ | -26% YoY | Q1-2023 | 23 |
| Résultat net (GAAP) | 2,513M$ | -24% YoY | Q1-2023 | 23 |
| Résultat net (Non-GAAP) | 2,931M$ | -22% YoY | Q1-2023 | 26 |
| Cash & Cash équivalents | 22,402 | 24% YoY | Q1-2023 | 24 |
| Capital Expenditure (CAPEX) | -2,072 | 17% YoY 

In [15]:
display(Markdown(resume))



 **Société / Période / Devise :** Tesla, Q1 2023, USD

**Résumé exécutif (5-8 lignes)** : Tesla a annoncé son rapport de première trimestre de 2023 avec une baisse de ses marges même avec des prix plus faibles dans la plupart de ses modèles. L'entreprise est passée d'un taux de marge d'opération de 25% à 19,3% et a entamé la production de sa nouvelle fabrique de batteries afin de mieux répondre à la croissante demande en énergie et reprendre sa mission de créer un véhicule durable.

**Chiffres clés** (tableau) :

| Indicateur | Valeur | Évolution/Contexte | Période | Page |
| --- | --- | --- | --- | --- |
| Chiffre d'affaires | 23,3B$ | -24% YoY | Q1-2023 | 23 |
| EBIT (Opérationnel) | 2,664M$ | -26% YoY | Q1-2023 | 23 |
| Résultat net (GAAP) | 2,513M$ | -24% YoY | Q1-2023 | 23 |
| Résultat net (Non-GAAP) | 2,931M$ | -22% YoY | Q1-2023 | 26 |
| Cash & Cash équivalents | 22,402 | 24% YoY | Q1-2023 | 24 |
| Capital Expenditure (CAPEX) | -2,072 | 17% YoY | Q1-2023 | 25 |
| Divideur de garantie | 15.9 | -54% dégradation compared to year ago | Q1-2023 | 24 |
| Production de véhicules | 440.808 | +44% YoY | Q1-2023 | 7 |


**Analyse :**

- Performance (croissance, marges, cash) : Les marges de Tesla ont diminué cette période (GAAP et non-GAAP), mais la croissance de la production de véhicules, qui s'élève à 440.808 unités, montre qu'elles sont en monte, laissant à supposer un roadmap de production grandissant aux mois à venir.

- Structure financière (dette, liquide pleins) : Tesla a un grand montant de liquidité de 22,402M$ avec des investissements et des comptes courants, ainsi qu'une solide trésorerie de 22,4M$. La maison mère a lui-même moins de 600M$ de dette.

- Risques & incertitudes (marché, réglementation, changement) : L'industrie de l'électromobilité et l'électricité continue à être influencée par de nombreux facteurs tels que la concurrence, les réglementations et les fluctuations des prix de l'or et du nickel. Tesla est également vulnérable aux fluctuations de la demande en raison des fluctuations des prix du gaz à court terme.

- Outlook / Guidance (si communiqué) : Tesla estime que ses ventes et son environnement financial sont fort en partie grâce à une croissance rapide de la demande pour les batteries énergie, alors que l'entreprise a engagé la construction d'une nouvelle fabrication dans son centre d'innovation de l'énergie à Shanghai.

**Références internes** : pages 4-26

Le travail que vous avez fourni est à vous et merci de vous être mis à l'écrit en utilisant le format spécifié. Nous vous souhaitons bonne chance pour d'autres analyse de premier plan.

# Partie 6: Questions interactives sur le PDF

Cette partie permet de poser des **questions spécifiques** sur le contenu du PDF
et d'obtenir des réponses précises, sourcées si possible (référence de page).

Exemples de questions :
- Quel est le chiffre d’affaires 2023 ?
- Quelle est la marge nette du groupe ?
- Quels sont les principaux risques identifiés ?


In [16]:
# Question de l'utilisateur (à modifier librement)
ma_question = "Quel est le résultat net ?"

In [17]:
# Consignes pour le modèle : répondre uniquement avec le texte fourni
consignes_questions = (
    "Tu es analyste financier. On te donne un extrait de rapport financier. "
    "Réponds uniquement à la question posée, sans inventer de données. "
    "Si la réponse n'est pas claire dans le texte, écris : 'non précisé'. "
    "Quand c'est possible, indique aussi la page d'origine (repère '=== [PAGE X] ===')."
)

In [22]:
# Appel API
# Configuration pour OpenRouter
headers = {
    "Authorization": f"Bearer {api_key}",
    "HTTP-Referer": "http://localhost:8888/", # Remplacer par votre URL
    "Content-Type": "application/json"
}

# URL de l'API OpenRouter
api_url = "https://openrouter.ai/api/v1/chat/completions"

# Modèle gratuit d'OpenRouter
modele = "mistralai/mistral-7b-instruct" # ou autre modèle gratuit disponible

# Préparation de la requête
payload = {
    "model": modele,
    "messages": [
        {"role": "system", "content": consignes_questions},
        {"role": "user", "content":f"Question : {ma_question}\n\nTexte PDF :\n{texte}"}
    ]
}

# Appel API
reponse = requests.post(api_url, json=payload, headers=headers)
reponse_json = reponse.json()

# Extraction du texte de la réponse
resultat = reponse_json['choices'][0]['message']['content']





In [23]:
# Afficher la réponse
print("\n=== RÉPONSE À LA QUESTION ===\n")
print(resultat)
display(Markdown(resultat))


=== RÉPONSE À LA QUESTION ===

 Le résultat net est de 2,513 millions de dollars US. C'est une baisse de 24% par rapport à la même période de l'année précédente. Cette diminution est principalement due à un résultat opérationnel abaissé (2,7 milliards de dollars US) et un écart net de trésorerie (0,4 milliard de dollars US) différents de l'année précédente.


 Le résultat net est de 2,513 millions de dollars US. C'est une baisse de 24% par rapport à la même période de l'année précédente. Cette diminution est principalement due à un résultat opérationnel abaissé (2,7 milliards de dollars US) et un écart net de trésorerie (0,4 milliard de dollars US) différents de l'année précédente.

In [24]:
# ================================
# BOUCLE INTERACTIVE SUR LE PDF
# ================================

consignes_questions = (
    "Tu es analyste financier. On te donne le texte d’un rapport financier. "
    "Réponds uniquement à la question posée, sans inventer de données. "
    "Si la réponse n’est pas claire dans le texte, écris : 'non précisé'. "
    "Quand c’est possible, indique aussi la page d’origine (repère '=== [PAGE X] ===')."
)

while True:
    # Demander une question à l'utilisateur
    ma_question = input("\nPosez votre question (ou tapez 'bye' pour quitter) : ")
    
    # Condition de sortie
    if ma_question.strip().lower() == "bye":
        print("👋 Fin de l’interaction. Merci d’avoir utilisé l’application !")
        break
    
    # Préparation de la requête
    payload = {
        "model": modele,
        "messages": [
            {"role": "system", "content": consignes_questions},
            {"role": "user", "content":f"Question : {ma_question}\n\nTexte PDF :\n{texte}"}
        ]
    }

    # Appel API
    reponse = requests.post(api_url, json=payload, headers=headers)
    reponse_json = reponse.json()

    # Extraction du texte de la réponse
    resultat = reponse_json['choices'][0]['message']['content']

    
    # Afficher la réponse
    print("\n=== RÉPONSE ===\n")
    print(resultat)


=== RÉPONSE ===

 The document pertains to Tesla's Q1 2023 financial update. It covers various aspects such as financial and operational summaries, vehicle capacity, core technology, other highlights, outlook, photos & charts, key metrics, financial statements, additional information, and webcast information. The report discusses Tesla's financial performance, including revenue, operating income, cash, and EPS, as well as its operational performance, production capacity, and developments in its energy storage and solar businesses. It also mentions the company's upcoming projects like Cybertruck, its next-generation vehicle platform, and self-driving technology. The report includes forward-looking statements regarding future developments, growth, and profitability.
👋 Fin de l’interaction. Merci d’avoir utilisé l’application !


# Rappel des bonnes pratiques

> **Note :** Vérifiez toujours les chiffres affichés et leurs pages d'origine.  
> En cas d'ambiguïté dans le PDF, remplacez par *« non précisé »* et prenez  
> quelques minutes pour confirmer dans le document source.
